In [1]:
import pandas as pd
import numpy as np
# import ujson as json
import time, datetime
import torch, nltk, re, random
from nltk.stem.porter import *
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
import data
import ujson as json

import warnings
warnings.filterwarnings("ignore", category=UserWarning) 

## 1. Create Dataframe for Parler Dataset

In [2]:
records = map(json.loads, open('datasets/parler_pretrain.ndjson'))
df = pd.DataFrame.from_records(records)
len_orig = len(df)
df = df[['body']]
df.replace('', np.nan, inplace=True)
df.dropna(inplace=True)
print("{}/{} ({:2f})% of samples are non-empty".format(len(df), len_orig, len(df)/len_orig*100))
df.head(10)

636417/1094836 (58.128980)% of samples are non-empty


,body
18,🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸RESPECT!🇺🇸🇺🇸...
204,Echo!
217,🏴󠁧󠁢󠁥󠁮󠁧󠁿🇬🇧👍
242,ALL DO.. I WILL WATCH WHEN THEY GIVE HIM THE L...
258,“STOP ALL Testing” Says Top Pathologist & COVI...
279,SEEING THAT HAPPEN WILL BEGIN A NEW CHAPTER FO...
287,"Wow, don’t mess with her, my hero!!!"
328,Banking Insider Exposes Covid-19 Mark of the B...
334,Never did and never will watch these cackling ...
346,AND BLACKS SOLD BLACKS TO THE WORLD !!


In [13]:
from transformers import BertTokenizer, BertForPreTraining, BertForMaskedLM

sentences = df['body'].values.tolist()[:10]
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 1. Create inputs, i.e. tokenized sentences
inputs = tokenizer(sentences, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
print(inputs.keys())

# 2. Create labels: a copy of input_ids, i.e. tokenized input
inputs['labels'] = inputs.input_ids.detach().clone()
print(inputs.keys())

# 3. Mask 15% of words in labels
rand = torch.rand(inputs.input_ids.shape)
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)
            # 101: input_id for [CLS] | 102: input_id for [SEP] | 0: input_id for [PAD]
# Select positions to be masked
selection = [torch.flatten(mask_arr[i].nonzero()).tolist() \
             for i in range(inputs.input_ids.shape[0])]
# Mask-out selected positions
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103 # input_id for [MASK]
print(inputs.input_ids[:10])


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
tensor([[  101,   100,   999,  ...,     0,     0,     0],
        [  101,  9052,   999,  ...,     0,     0,     0],
        [  101,   100,   102,  ...,     0,     0,     0],
        ...,
        [  101,   103, 25297,  ...,     0,     0,     0],
        [  101,  2196,   103,  ...,     0,     0,     0],
        [  101,  1998, 10823,  ...,     0,     0,     0]])


In [30]:
dataset = TensorDataset(inputs.input_ids, inputs.attention_mask, inputs.labels)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)


In [31]:
from transformers import AdamW
from tqdm import tqdm  

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.to(device)
model.train()
optim = AdamW(model.parameters(), lr=5e-5)

epochs = 2

for epoch in range(epochs):
    for batch in tqdm(dataloader):
        optim.zero_grad()
        input_id, input_mask, label = \
            batch[0].to(device), batch[1].to(device), batch[2].to(device)
        outputs = model(input_ids = input_id, attention_mask=input_mask,
                        labels=label)

        loss = outputs.loss
        loss.backward()
        optim.step()

        tqdm(dataloader).set_description(f'Epoch {epoch}')
        tqdm(dataloader).set_postfix(loss=loss.item())

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: CUDA out of memory. Tried to allocate 90.00 MiB (GPU 0; 10.76 GiB total capacity; 9.20 GiB already allocated; 9.44 MiB free; 9.28 GiB reserved in total by PyTorch)